In [ ]:
### If Google Colab
!git config --global user.email "sjs.shamsi@gmail.com"
!git config --global user.name "SJShamsi"

from google.colab import drive
drive.mount('/content/drive')

Thesis_Files = '/content/drive/MyDrive/Thesis_Files/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### Importing common items
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')
plt.style.use('seaborn-v0_8-colorblind')
plt.rcParams['axes.grid'] = False

In [ ]:
import sys
# sys.path.insert(0, '/home/shoaib/Thesis/dmdt_Analysis/')
sys.path.insert(0, Thesis_Files + 'Thesis/dmdt_Analysis/')
sys.path.insert(0, Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/')

from fastai_fit_model import *

!pip install fbpca &> /dev/null
from fbpca_implementation import *

In [ ]:
dmdt_histograms_50x50_linear_bins_file = Thesis_Files + 'Thesis/dmdt_Analysis/Histograms/dmdt_by_Name_Band_50x50_linear_bins.pkl'
dmdt_histograms_50x50_equivalent_bins_file = Thesis_Files + 'Thesis/dmdt_Analysis/Histograms/dmdt_by_Name_Band_50x50_equivalent_bins.pkl'
dmdt_histograms_21x23_Mahabal_bins_file = Thesis_Files + 'Thesis/dmdt_Analysis/Histograms/dmdt_histograms_21x23_Mahabal_bins.pkl'

### the following are equivalent bins
dm_bins_equivalent = np.load(Thesis_Files + 'Thesis/dmdt_Analysis/Bins/equivalent_bins_50x50_QSOs/QSO_dm_bins.npy')
dt_bins_equivalent = np.load(Thesis_Files + 'Thesis/dmdt_Analysis/Bins/equivalent_bins_50x50_QSOs/QSO_dt_bins.npy')

### the following are linear bins
nxbins, nybins = 51, 51
dm_bins_linear = np.linspace(-1, 1, nybins)
dt_bins_linear = np.linspace(0, 1160, nxbins)

### Following are the Mahabal bins
dm_bins_Mahabal = np.load(Thesis_Files + 'Thesis/dmdt_Analysis/Bins/dm_-1_1_Mahabal_bins_22.npy')
dt_bins_Mahabal = np.load(Thesis_Files + 'Thesis/dmdt_Analysis/Bins/dt_0_1160_Mahabal_bins_24.npy')

In [ ]:
def get_FBPA_background(histogram_list, operation):
    r_histograms = np.array([arr[:, :, 0] for arr in histogram_list])
    g_histograms = np.array([arr[:, :, 1] for arr in histogram_list])

    M_r = np.stack([arr.flatten() for arr in r_histograms], axis=1)
    M_g = np.stack([arr.flatten() for arr in g_histograms], axis=1)

    L_r, S_r, examples =  pcp(M_r, maxiter=7, k=10, example_column=1000)
    L_g, S_g, examples =  pcp(M_g, maxiter=7, k=10, example_column=1000)

    g_max_background = np.array([operation(L_g[i, :]) for i in range(len(L_g[:, 0]))])
    r_max_background = np.array([operation(L_r[i, :]) for i in range(len(L_r[:, 0]))])

    r_max_background = r_max_background.reshape(histogram_list[0][:, :, 0].shape)
    g_max_background = g_max_background.reshape(histogram_list[0][:, :, 0].shape)

    background = np.zeros_like(histogram_list[0])
    background[:, :, 0] = r_max_background
    background[:, :, 1] = g_max_background
    return background

## Equivalent Bins

In [ ]:
dmdt_df = pd.read_pickle(dmdt_histograms_50x50_equivalent_bins_file)
dmdt_df = dmdt_df[dmdt_df['type'].isin(['Q', 'QR', 'QX'])]
dmdt_df = dmdt_df[dmdt_df['band'] == 'rg']
dmdt_df.head()

,name,type,band,n_good_det,hist_p_scale,histogram
6565,1WGA J16290+3724A,Q,rg,"[774, 725]","[298433, 261893]","[[[0.0, 1.1455059890871462e-05, 0.0], [0.0, 2.672847307870008e-05, 0.0], [0.0, 0.0, 0.0], [0.0, 3.8183532969571546e-06, 0.0], [0.0, 0.0, 0.0], [0.0, 1.1455059890871462e-05, 0.0], [0.0, 1.909176648478577e-05, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.909176648478577e-05, 0.0], [0.0, 7.636706593914309e-06, 0.0], [0.0, 0.0, 0.0], [3.3508358660067755e-06, 1.1455059890871462e-05, 0.0], [0.0, 2.672847307870008e-05, 0.0], [3.3508358660067755e-06, 2.672847307870008e-05, 0.0], [0.0, 4.582023956348585e-05, 0.0], [2.0105015196040652e-05, 8.782212583001455e-05, 0.0], [0.00015078..."
6566,2MASX J15371580+5829336,Q,rg,"[1354, 1311]","[913473, 856477]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."
6567,2QZ J095248.7+004930,Q,rg,"[204, 167]","[20633, 13785]","[[[0.0, 7.254261878853827e-05, 0.0], [9.693209906460525e-05, 0.0007979688066739209, 0.0], [0.00014539814859690787, 0.00014508523757707654, 0.0], [9.693209906460525e-05, 0.0004352557127312296, 0.0], [0.0, 0.0007254261878853826, 0.0], [0.00014539814859690787, 7.254261878853827e-05, 0.0], [4.846604953230262e-05, 0.00014508523757707654, 0.0], [0.0001938641981292105, 0.0003627130939426913, 0.0], [0.00029079629719381574, 0.0005077983315197679, 0.0], [0.0, 0.0005803409503083062, 0.0], [0.0, 0.0003627130939426913, 0.0], [4.846604953230262e-05, 0.0006528835690968443, 0.0], [4.846604953230262e-05, 0..."
6568,2QZ J095543.3+010536,Q,rg,"[124, 84]","[7540, 3412]","[[[0.0, 0.00029308323563892143, 0.0], [0.0003978779840848806, 0.00029308323563892143, 0.0], [0.001326259946949602, 0.00029308323563892143, 0.0], [0.0007957559681697613, 0.00029308323563892143, 0.0], [0.0010610079575596816, 0.0008792497069167644, 0.0], [0.0005305039787798408, 0.0008792497069167644, 0.0], [0.000663129973474801, 0.00029308323563892143, 0.0], [0.001326259946949602, 0.0020515826494724504, 0.0], [0.0009283819628647215, 0.0020515826494724504, 0.0], [0.001326259946949602, 0.0029308323563892145, 0.0], [0.0007957559681697613, 0.0, 0.0], [0.0017241379310344827, 0.0008792497069167644,..."
6569,2QZ J100058.3+000530,Q,rg,"[130, 114]","[8372, 6432]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.00011944577161968466, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0001554726368159204, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.00011944577161968466, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0004664179104477612, 0.0], [0.00023889154323936931, 0.0001554726368159204, 0.0], [0.0, 0.0001554726368159204, 0.0], [0.00011944577161968466, 0.0003109452736318408, 0.0], [0.000716674629718108, 0.0009328358208955224, 0.0], [0.0013139034878165313, 0.002021144278606965, 0.0], [0.000836120401337..."


### We'll be calculating class-specific backgrounds first.

In [ ]:
Q_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.median)
QR_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.median)
QX_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.median)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_median_background_FBPCA_50x50_equivalent_bins.npy', Q_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_median_background_FBPCA_50x50_equivalent_bins.npy', QR_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_median_background_FBPCA_50x50_equivalent_bins.npy', QX_median_background)

In [ ]:
Q_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.mean)
QR_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.mean)
QX_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.mean)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_mean_background_FBPCA_50x50_equivalent_bins.npy', Q_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_mean_background_FBPCA_50x50_equivalent_bins.npy', QR_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_mean_background_FBPCA_50x50_equivalent_bins.npy', QX_mean_background)

In [ ]:
Q_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.max)
QR_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.max)
QX_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_max_background_FBPCA_50x50_equivalent_bins.npy', Q_max_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_max_background_FBPCA_50x50_equivalent_bins.npy', QR_max_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_max_background_FBPCA_50x50_equivalent_bins.npy', QX_max_background)

### Calculating imbalanced QSO backgrounds

In [ ]:
QSO_median_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.median)
QSO_mean_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.mean)
QSO_max_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_median_background_FBPCA_50x50_equivalent_bins.npy', QSO_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_mean_background_FBPCA_50x50_equivalent_bins.npy', QSO_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_max_background_FBPCA_50x50_equivalent_bins.npy', QSO_max_background)

### Calculating balanced QSO backgrounds

In [ ]:
n_samples = dmdt_df['type'].value_counts().values[-1]
dmdt_balanced_df = dmdt_df.groupby('type', group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))
dmdt_balanced_df

<ipython-input-11-ae843a608ea8>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dmdt_balanced_df = dmdt_df.groupby('type', group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))


,name,type,band,n_good_det,hist_p_scale,histogram
16783,SDSS J135054.45+441539.1,Q,rg,"[549, 554]","[149333, 152135]","[[[0.0001004466527827071, 2.629243763762448e-05, 0.0], [0.00045535815928160554, 0.000604726065665363, 0.0], [0.0005022332639135355, 0.00042067900220199166, 0.0], [0.00037500083705543984, 0.0002497781575574325, 0.0], [0.00046205460280045267, 0.0003089361422420876, 0.0], [0.000381697280574287, 0.0001971932822821836, 0.0], [0.00017410753149002563, 0.00034837479869852435, 0.0], [0.0002544648537161913, 0.00040095967397377327, 0.0], [0.0002477684101973442, 0.00023663193873862032, 0.0], [0.00030803640186696844, 0.00024320504814802642, 0.0], [0.0001339288703769428, 0.00021691261051040196, 0.0], [0..."
19359,SDSS J145905.65+473249.9,Q,rg,"[646, 563]","[207730, 157752]","[[[0.0, 5.071251077640854e-05, 0.0], [1.4441823520916574e-05, 1.9017191541153202e-05, 0.0], [3.369758821547201e-05, 2.535625538820427e-05, 0.0], [4.8139411736388584e-06, 1.2678127694102135e-05, 0.0], [2.888364704183315e-05, 0.00010142502155281708, 0.0], [9.627882347277717e-06, 0.00012678127694102135, 0.0], [1.4441823520916574e-05, 8.240783001166388e-05, 0.0], [4.8139411736388584e-06, 8.240783001166388e-05, 0.0], [9.627882347277717e-06, 3.169531923525534e-05, 0.0], [9.627882347277717e-06, 3.169531923525534e-05, 0.0], [4.8139411736388584e-06, 8.874689385871494e-05, 0.0], [9.627882347277717e-..."
11605,SDSS J104209.42+400720.0,Q,rg,"[277, 251]","[38157, 31335]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."
25938,SDSS J234101.58+143134.0,Q,rg,"[291, 267]","[42113, 35418]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."
24487,SDSS J171017.27+353054.8,Q,rg,"[765, 770]","[291621, 295494]","[[[0.0, 1.0152490405896566e-05, 0.0], [0.0, 1.0152490405896566e-05, 0.0], [0.0, 6.7683269372643775e-06, 0.0], [0.0, 0.0, 0.0], [0.0, 6.7683269372643775e-06, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 2.368914428042532e-05, 0.0], [0.0, 6.7683269372643775e-06, 0.0], [0.0, 2.707330774905751e-05, 0.0], [0.0, 4.399412509221846e-05, 0.0], [0.0, 4.399412509221846e-05, 0.0], [1.028732498688366e-05, 2.707330774905751e-05, 0.0], [0.0, 2.0304980811793132e-05, 0.0], [0.0, 6.768326937264377e-05, 0.0], [6.8582166579224405e-06, 0.0001184457214021266, 0.0], [2.7432866631689762e-05, 0.000192897317712034..."
...,...,...,...,...,...,...
30118,SDSS J121722.75+614628.5,QX,rg,"[644, 462]","[198816, 106134]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.884410273804813e-05, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 9.422051369024064e-06, 0.0], [0.0, 0.0, 0.0], [0.0, 1.884410273804813e-05, 0.0], [0.0, 9.422051369024064e-06, 0.0], [0.0, 2.8266154107072193e-05, 0.0], [0.0, 0.0002355512842256016, 0.0], [5.0297762755512636e-06, 0.0003109276951777941, 0.0], [0.0001106550780621278, 0.0007820302636289973, 0.0], [0.0001458635119

In [ ]:
QSO_median_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.median)
QSO_mean_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.mean)
QSO_max_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_median_background_FBPCA_50x50_equivalent_bins.npy', QSO_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_mean_background_FBPCA_50x50_equivalent_bins.npy', QSO_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_max_background_FBPCA_50x50_equivalent_bins.npy', QSO_max_background)

## Linear Bins

### Class Specific Backgrounds

In [ ]:
dmdt_df = pd.read_pickle(dmdt_histograms_50x50_linear_bins_file)
dmdt_df = dmdt_df[dmdt_df['type'].isin(['Q', 'QR', 'QX'])]
dmdt_df = dmdt_df[dmdt_df['band'] == 'rg']
dmdt_df.head()

,name,type,band,n_good_det,hist_p_scale,histogram
6565,1WGA J16290+3724A,Q,rg,"[774, 725]","[299151, 262450]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."
6566,2MASX J15371580+5829336,Q,rg,"[1354, 1311]","[915981, 858705]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."
6567,2QZ J095248.7+004930,Q,rg,"[204, 167]","[20706, 13847]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 7.221780891167762e-05, 0.0]..."
6568,2QZ J095543.3+010536,Q,rg,"[124, 84]","[7567, 3418]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0001321527686005022, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0002925687536571094, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0001321527686005022, 0.0002925687536571094, 0.0], [0.0, 0.0002925687536571094, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
6569,2QZ J100058.3+000530,Q,rg,"[130, 114]","[8385, 6437]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."


In [ ]:
Q_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.median)
QR_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.median)
QX_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.median)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_median_background_FBPCA_50x50_linear_bins.npy', Q_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_median_background_FBPCA_50x50_linear_bins.npy', QR_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_median_background_FBPCA_50x50_linear_bins.npy', QX_median_background)

In [ ]:
Q_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.mean)
QR_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.mean)
QX_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.mean)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_mean_background_FBPCA_50x50_linear_bins.npy', Q_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_mean_background_FBPCA_50x50_linear_bins.npy', QR_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_mean_background_FBPCA_50x50_linear_bins.npy', QX_mean_background)

In [ ]:
Q_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.max)
QR_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.max)
QX_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_max_background_FBPCA_50x50_linear_bins.npy', Q_max_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_max_background_FBPCA_50x50_linear_bins.npy', QR_max_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_max_background_FBPCA_50x50_linear_bins.npy', QX_max_background)

### Calculating imbalanced QSO Backgrounds

In [ ]:
QSO_median_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.median)
QSO_mean_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.mean)
QSO_max_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_median_background_FBPCA_50x50_linear_bins.npy', QSO_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_mean_background_FBPCA_50x50_linear_bins.npy', QSO_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_max_background_FBPCA_50x50_linear_bins.npy', QSO_max_background)

### Calculating balanced QSO Backgrounds

In [ ]:
n_samples = dmdt_df['type'].value_counts().values[-1]
dmdt_balanced_df = dmdt_df.groupby('type', group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))
dmdt_balanced_df.head()

<ipython-input-18-5642a29ce045>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dmdt_balanced_df = dmdt_df.groupby('type', group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))


,name,type,band,n_good_det,hist_p_scale,histogram
8809,SDSS J030128.16+005804.0,Q,rg,"[342, 296]","[58311, 43660]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."
18489,SDSS J143927.64+491602.0,Q,rg,"[1234, 1134]","[760761, 642404]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.556652822834229e-06, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]..."
24204,SDSS J165943.67+314915.7,Q,rg,"[1169, 1184]","[682582, 697754]","[[[2.9300508949840457e-06, 2.0064377989950612e-05, 0.0], [5.860101789968091e-06, 5.732679425700175e-06, 0.0], [1.4650254474920228e-06, 2.0064377989950612e-05, 0.0], [8.790152684952137e-06, 2.1497547846375657e-05, 0.0], [1.6115279922412253e-05, 4.299509569275131e-06, 0.0], [5.860101789968091e-06, 1.0032188994975306e-05, 0.0], [1.4650254474920228e-06, 7.165849282125219e-06, 0.0], [4.3950763424760685e-06, 4.299509569275131e-06, 0.0], [1.4650254474920228e-06, 1.4331698564250437e-06, 0.0], [1.4650254474920228e-06, 1.4331698564250437e-06, 0.0], [0.0, 2.8663397128500874e-06, 0.0], [0.0, 0.0, 0.0]..."
14361,SDSS J123701.16+641629.0,Q,rg,"[255, 248]","[32385, 30628]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."
19336,SDSS J145838.89+395334.3,Q,rg,"[515, 494]","[132355, 121771]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0],..."


In [ ]:
QSO_median_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.median)
QSO_mean_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.mean)
QSO_max_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_median_background_FBPCA_50x50_linear_bins.npy', QSO_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_mean_background_FBPCA_50x50_linear_bins.npy', QSO_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_max_background_FBPCA_50x50_linear_bins.npy', QSO_max_background)

## Mahabal Bins

### Class Specific Backgrounds

In [ ]:
dmdt_df = pd.read_pickle(dmdt_histograms_21x23_Mahabal_bins_file)
dmdt_df = dmdt_df[dmdt_df['type'].isin(['Q', 'QR', 'QX'])]
dmdt_df = dmdt_df[dmdt_df['band'] == 'rg']
dmdt_df.head()

,name,type,band,n_good_det,hist_p_scale,histogram
6565,1WGA J16290+3724A,Q,rg,"[774, 725]","[299151, 262450]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 3.8102495713469234e-06, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 7.620499142693847e-06, 0.0], [0.0002674234751012031, 2.6671746999428462e-05, 0.0], [0.00034430772419279896, 0.0008268241569822823, 0.0], [0.00029082302917255836, 0.0004610401981329777, 0.0]], [[3.3427934387650386e-06, 3.810249571..."
6566,2MASX J15371580+5829336,Q,rg,"[1354, 1311]","[915981, 858705]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0..."
6567,2QZ J095248.7+004930,Q,rg,"[204, 167]","[20706, 13847]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 7.221780891167762e-05, 0.0], [0.0, 0.0, 0.0], [4.829518014102193e-05, 0.0, 0.0], [0.0, 0.0002888712356467105, 0.0], [0.0, 0.00014443561782335524, 0.0], [4.829518014102193e-05, 7.221780891167762e-05, 0.0], [4.829518014102193e-05, 0.0012999205604101972, 0.0], [0.0, 0.0009388315158518091, 0.0], [0.0, 0.0006499602802050986, 0.0], [0.0005795421616922632, 0.0032498014010..."
6568,2QZ J095543.3+010536,Q,rg,"[124, 84]","[7567, 3418]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0002925687536571094, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0002925687536571094, 0.0], [0.0, 0.0, 0.0], [0.0010572221488040175, 0.0005851375073142189, 0.0], [0.0006607638430025109, 0.0002925687536571094, 0.0], [0.0001321527686005022, 0.0002925687536571094, 0.0], [0.002378749834809039, 0.002925687536571094, 0.0], [0.005550416281221091, 0.0052662375658279695, 0.0], [0.0013215276860050217, ..."
6569,2QZ J100058.3+000530,Q,rg,"[130, 114]","[8385, 6437]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.00015535187199005747, 0.0], [0.0, 0.00031070374398011494, 0.0], [0.0014311270125223613, 0.0006214074879602299, 0.0], [0.0019081693500298152, 0.004660556159701725, 0.0], [0.0004770423375074538, 0.00046605561597017243, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0..."


In [ ]:
Q_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.median)
QR_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.median)
QX_median_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.median)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_median_background_FBPCA_21x23_Mahabal_bins.npy', Q_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_median_background_FBPCA_21x23_Mahabal_bins.npy', QR_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_median_background_FBPCA_21x23_Mahabal_bins.npy', QX_median_background)

In [ ]:
Q_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.mean)
QR_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.mean)
QX_mean_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.mean)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_mean_background_FBPCA_21x23_Mahabal_bins.npy', Q_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_mean_background_FBPCA_21x23_Mahabal_bins.npy', QR_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_mean_background_FBPCA_21x23_Mahabal_bins.npy', QX_mean_background)

In [ ]:
Q_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'Q']['histogram'].to_numpy(), np.max)
QR_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QR']['histogram'].to_numpy(), np.max)
QX_max_background = get_FBPA_background(dmdt_df[dmdt_df['type'] == 'QX']['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/Q_max_background_FBPCA_21x23_Mahabal_bins.npy', Q_max_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_max_background_FBPCA_21x23_Mahabal_bins.npy', QR_max_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QR_max_background_FBPCA_21x23_Mahabal_bins.npy', QX_max_background)

### Calculating imbalanced QSO Backgrounds

In [ ]:
QSO_median_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.median)
QSO_mean_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.mean)
QSO_max_background = get_FBPA_background(dmdt_df['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_median_background_FBPCA_21x23_Mahabal_bins.npy', QSO_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_mean_background_FBPCA_21x23_Mahabal_bins.npy', QSO_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_imbalanced_max_background_FBPCA_21x23_Mahabal_bins.npy', QSO_max_background)

### Calculating balanced QSO Backgrounds

In [ ]:
n_samples = dmdt_df['type'].value_counts().values[-1]
dmdt_balanced_df = dmdt_df.groupby('type', group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))
dmdt_balanced_df.head()

<ipython-input-35-5642a29ce045>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dmdt_balanced_df = dmdt_df.groupby('type', group_keys=False).apply(lambda x: x.sample(n=min(len(x), n_samples)))


,name,type,band,n_good_det,hist_p_scale,histogram
21653,SDSS J155628.52+523014.1,Q,rg,"[638, 541]","[203188, 146061]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [4.921550485264878e-06, 0.0, 0.0], [0.0, 0.0, 0.0], [1.4764651455794633e-05, 2.0539363690512867e-05, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [2.460775242632439e-05, 1.3692909127008579e-05, 0.0], [0.0, 6.8464545635042896e-06, 0.0], [5.905860582317853e-05, 9.585036388906005e-05, 0.0], [5.4137055337913654e-05, 0.00011638972757957292, 0.0], [8.366635824950292e-05, 0.0001369290912700858, 0.0], [0.0001771758174695356, 6.161809107153861e-05, 0.0], [0.00015256806504321122, 8.900..."
16848,SDSS J135304.05+081410.0,Q,rg,"[265, 247]","[34980, 30345]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.00013181743285549513, 0.0], [0.0, 9.886307464162135e-05, 0.0], [0.0, 0.00013181743285549513, 0.0], [0.0, 9.886307464162135e-05, 0.0], [0.0, 3.295435821387378e-05, 0.0], [2.858776443682104e-05, 0.00013181743285549513, 0.0], [0.0, 0.0007909045971329708, 0.0], [0.0001429388221841052, 0.002932937881034767, 0.0], [0.0004288164665523156, 0.007810182896..."
18558,SDSS J144115.17+481600.1,Q,rg,"[621, 820]","[192510, 335790]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0..."
21458,SDSS J155145.34+571738.4,Q,rg,"[764, 501]","[291466, 125007]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.5999104050173192e-05, 0.0], [0.0, 0.0, 0.0], [0.0, 1.5999104050173192e-05, 0.0], [0.0, 3.1998208100346384e-05, 0.0], [0.0, 0.0, 0.0], [0.0, 2.3998656075259786e-05, 0.0], [0.0, 7.999552025086596e-06, 0.0], [0.0, 0.0001439919364515587, 0.0], [0.0, 0.0001919892486020783, 0.0], [0.0, 0.0003839784972041566, 0.0], [0.0, 0.0005359699856808019, 0.0], [0.0, 0.00033598118505363697, 0.0], [3.4309319097253195e-06, 0.0016079099570424056, 0.0], [0.0, 0.0014559184685657602, 0.0..."
13630,SDSS J120848.54+555804.0,Q,rg,"[596, 425]","[177310, 90037]","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [1.1279679657097739e-05, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 2.221309017403956e-05, 0.0], [0.0, 0.0, 0.0], [0.0, 2.221309017403956e-05, 0.0], [0.0, 0.0, 0.0], [1.1279679657097739e-05, 3.331963526105934e-05, 0.0], [0.0, 3.331963526105934e-05, 0.0], [0.0, 2.221309017403956e-05, 0.0], [1.6919519485646606e-05, 6.663927052211869e-05, 0.0], [1.6919519485646606e-05, 0.0004886879838288704, 0.0], [4.5118718628390956e-05, 0.00053311..."


In [ ]:
QSO_median_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.median)
QSO_mean_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.mean)
QSO_max_background = get_FBPA_background(dmdt_balanced_df['histogram'].to_numpy(), np.max)

np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_median_background_FBPCA_21x23_Mahabal_bins.npy', QSO_median_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_mean_background_FBPCA_21x23_Mahabal_bins.npy', QSO_mean_background)
np.save(Thesis_Files + 'Thesis/dmdt_Analysis/RPCA/QSO_balanced_max_background_FBPCA_21x23_Mahabal_bins.npy', QSO_max_background)